# Imports

In [1]:
import os
import gc
import pickle

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

from flair.data import TaggedCorpus
from flair.data_fetcher import  NLPTaskDataFetcher, NLPTask

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
import torch

# Try to modify learned represention and use partial model to make predictionsm

In [18]:
tagger = SequenceTagger.load('chunk')

2019-03-23 10:41:04,089 loading file /home/snie/.flair/models/en-chunk-conll2000-v0.2.pt


In [76]:
tagger

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(227, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=227, bias=True)
      )
    )
    (list_embedding_1): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(227, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=227, bias=True)
      )
    )
  )
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=45, bias=True)
)

In [75]:
help(tagger.predict)

Help on method predict in module flair.models.sequence_tagger_model:

predict(sentences:Union[List[flair.data.Sentence], flair.data.Sentence], mini_batch_size=32, verbose=False) -> List[flair.data.Sentence] method of flair.models.sequence_tagger_model.SequenceTagger instance
    Predicts the labels/tags for the given list of sentences. The labels/tags are added directly to the
    sentences.



In [74]:
tagger.predict(Sentence('a b c'))

[Sentence: "a b c" - 3 Tokens]

In [31]:
t = tagger.linear

In [47]:
a = torch.rand(512)

In [51]:
t.cuda()

Linear(in_features=512, out_features=45, bias=True)

In [55]:
a = a.cuda()

In [68]:
torch.save(t, './partial_models/flair_chunck_linear_layer.pt')

In [69]:
model = torch.load('./partial_models/flair_chunck_linear_layer.pt')

In [70]:
model

Linear(in_features=512, out_features=45, bias=True)

In [58]:
t.forward(a)

tensor([-0.3074,  3.0060,  3.7630, -4.7441,  3.5282, -0.3419,  1.1387,  1.6184,
        -0.5954,  1.4467, -2.0640, -1.3344, -0.5230,  5.2574,  1.7556,  3.7739,
         4.8887,  0.3100, -1.7064, -1.4568, -3.6022, -3.5891, -6.6334, -1.9965,
        -0.4712,  3.6837,  0.0715,  0.4407,  1.6095, -0.6557, -0.7850, -0.1129,
         1.0722, -0.0530, -1.8550, -2.8702,  0.5367, -1.2927,  0.6121, -1.5596,
        -0.6297, -1.0897, -0.3128, -0.2445,  0.2355], device='cuda:0',
       grad_fn=<AddBackward0>)

In [37]:
p.shape

torch.Size([45])

In [39]:
for p in t.parameters():
    print(p.shape)

torch.Size([45, 512])
torch.Size([45])


In [15]:
tagger

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_1): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=53, bias=True)
)

In [13]:
ner_tagger

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings()
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout()
  (locked_dropout): LockedDropout()
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=20, bias=True)
)

# Initialize Mongo database

In [80]:
from pymongo import MongoClient

client = MongoClient()

db = client['glvis_db']

In [48]:
len(list(db['flair_chunk'].find({'text':'in'})))

4125

In [103]:
term = 'in'
pipeline = {
    '$text': {'$search': f'{term}'}
}

len(list(db['flair_chunk'].find(pipeline)))

4657

# Database util

### Flattened val_collection

In [20]:
for rec in val_collection.find():
    break

In [10]:
rec['label']

{'value': 'POSITIVE', 'confidence': 1.0}

In [22]:
add_fields = [
    {'$addFields': {'sentiment': '$label.value', 'confidence': '$label.confidence'}},
    {'$out': 'flattened'}
]

In [24]:
flattened = db['flattened']

In [27]:
# delete label fields in the document store
flattened.update_many({}, {'$unset': {'label': ''}})

### Add index to val_collection

In [51]:
import pymongo

In [98]:
val_collection = db['glove_6b_300d']

In [99]:
val_collection.drop_index('word_text')

In [100]:
val_collection.create_index([('word', pymongo.TEXT)], default_language='none')

'word_text'

In [61]:
# Test search on the index
cur = val_collection.find({
    '$and': [
        {'$text': {'$search': 'happy'}}, 
        {'sentiment': 'NEGATIVE'},
        {'confidence': {'$eq': 1.0}}
    ]
})

In [60]:
len(list(cur))

404

### Play with database

In [3]:
def query(term):
    pipeline = {
        '$text': {'$search': term}
    }

    return list(val_collection.find(pipeline))

In [8]:
res = query('\"movie\""')

In [84]:
vectors = np.array([elm['val'] for elm in res])

In [98]:
mean = np.mean(vectors, axis=0)
std = np.mean(vectors, axis=0)
stats = [
    {
        'dim': i,
        'mean': val[0],
        'std': val[1]
    } for i, val in enumerate(zip(mean, std))
]

### Utilities

In [127]:
pipeline = [
    {"$group": {"_id": "$sentence", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1 }}}
]

In [128]:
res = val_collection.aggregate(pipeline, allowDiskUse=True)